<a href="https://colab.research.google.com/github/ijusplab/mineracao-processos/blob/main/MINERACAO_PROCESSOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demonstração Mineração de Processos**

## **Instruções**

**INSTRUÇÕES**

1) Execute a célula `CARREGAR FERRAMENTA`. 

2) Nos Relatórios Gerenciais PJe 1G (http://web.trf3.jus.br/relatorios-gerenciais), acesse o relatório `Relação do histórico de movimentações de processos`, escolha o órgão julgador e baixe o relatório em formato `csv`, salvando-o em uma pasta de fácil acesso no seu computador com o nome `log_eventos.csv`. 

3) Clique no ícone das pastinha aqui ao lado esquerdo. Você verá que uma barra lateral irá se abrir. Arraste o arquivo `log_eventos.csv` para lá para salvá-lo no Google Colab.

4) Depois que o arquivo `log_eventos.csv` estiver salvo no Google Colab, execute a célula `CARREGAR RELATÓRIO GERENCIAL`. 

5) Execute o 3º e 4º Passos para ver o mapa de processos e o mapa social. 


## **Ver Fluxo Processual**

In [ ]:
#@title 1º PASSO) **CARREGAR FERRAMENTA** { vertical-output: true }
MODO_DEPURACAO = False

!pip install pm4py

import re, os, json, csv, pprint, math
from google.colab import files
import pandas as pd
import numpy as np
import graphviz
import xlrd
import pm4py
from IPython.display import display, HTML

##
## Mensagens
##

class mensageria:
  __END      = '\33[0m'
  __BOLD     = '\33[1m'
  __ITALIC   = '\33[3m'
  __URL      = '\33[4m'
  __BLINK    = '\33[5m'
  __BLINK2   = '\33[6m'
  __SELECTED = '\33[7m'

  __BLACK  = '\33[30m'
  __RED    = '\33[31m'
  __GREEN  = '\33[32m'
  __YELLOW = '\33[33m'
  __BLUE   = '\33[34m'
  __VIOLET = '\33[35m'
  __BEIGE  = '\33[36m'
  __WHITE  = '\33[37m'

  __BLACKBG  = '\33[40m'
  __REDBG    = '\33[41m'
  __GREENBG  = '\33[42m'
  __YELLOWBG = '\33[43m'
  __BLUEBG   = '\33[44m'
  __VIOLETBG = '\33[45m'
  __BEIGEBG  = '\33[46m'
  __WHITEBG  = '\33[47m'

  __GREY    = '\33[90m'
  __RED2    = '\33[91m'
  __GREEN2  = '\33[92m'
  __YELLOW2 = '\33[93m'
  __BLUE2   = '\33[94m'
  __VIOLET2 = '\33[95m'
  __BEIGE2  = '\33[96m'
  __WHITE2  = '\33[97m'

  __GREYBG    = '\33[100m'
  __REDBG2    = '\33[101m'
  __GREENBG2  = '\33[102m'
  __YELLOWBG2 = '\33[103m'
  __BLUEBG2   = '\33[104m'
  __VIOLETBG2 = '\33[105m'
  __BEIGEBG2  = '\33[106m'
  __WHITEBG2  = '\33[107m'

  @classmethod
  def sucesso(cls, msg):
    print(f'{cls.__BLUE}{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def falha(cls, msg):
    print(f'{cls.__REDBG}{cls.__WHITE}{msg}{cls.__END}')

  @classmethod
  def alerta(cls, msg):
    print(f'{cls.__YELLOWBG}{msg}{cls.__END}')

  @classmethod
  def ok(cls, msg):
    print(f'{cls.__GREEN}{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def normal(cls, msg):
    print(f'{cls.__BOLD}{msg}{cls.__END}')

  @classmethod
  def mostrar_objeto(cls, obj):
    printer = pprint.PrettyPrinter(indent=4)
    printer.pprint(obj)

##
## Extração dos dados do sharepoint
##
class PMTools:
  _instance = None

  def __init__(self):
    self.__event_log = None
    self.__social_log = None
    self.__process_tree = None
    self.__social_tree = None
    self.__bpmn_model = None
    self.__dfg = None
    self.__dfg_social = None
    self.__start_activities = None
    self.__end_activities = None
    self.__start_resources = None
    self.__end_resources = None
    self.__num_events = 0
    self.__num_cases = 0

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def __to_datetime(self, xl_date_string):
    if not isinstance(xl_date_string, str):
      return xl_date_string
    if not xl_date_string:
      return xl_date_string
    date, time = xl_date_string.split(' ')
    date = date.split('/')[::-1]
    date = '-'.join(date)
    return f'{date} {time}'

  def __parse_df(self, df):
    columns = {'processo': 'case_id', 'tarefa': 'activity', 'entrada': 's_timestamp', 'saida': 'e_timestamp', 'usuario': 'resource'}
    df.rename(columns=columns, inplace=True)
    df['s_timestamp'] = df['s_timestamp'].apply(self.__to_datetime)
    df['e_timestamp'] = df['e_timestamp'].apply(self.__to_datetime)
    self.__num_events = len(df)
    self.__num_cases = len(df.case_id.unique())
    self.__event_log = pm4py.format_dataframe(df, case_id='case_id', activity_key='activity', timestamp_key='s_timestamp')
    self.__social_log = pm4py.format_dataframe(df, case_id='case_id', activity_key='resource', timestamp_key='s_timestamp')
    self.__process_tree = pm4py.discover_process_tree_inductive(self.__event_log)
    self.__social_tree = pm4py.discover_process_tree_inductive(self.__social_log)
    self.__bpmn_model = pm4py.convert_to_bpmn(self.__process_tree)
    self.__dfg, self.__start_activities, self.__end_activities = pm4py.discover_dfg(self.__event_log)
    self.__dfg_social, self.__start_resources, self.__end_resources = pm4py.discover_dfg(self.__social_log)

  def load(self):
    uploaded = files.upload()
    arquivos = list(uploaded.keys())
    print(arquivos)

    if len(arquivos) < 0:
      mensageria.falha('\n>>>> Nenhum arquivo selecionado')
      return

    src = arquivos[0]

    if not re.match('^.+\.(csv)$', src):
      os.remove(src)
      mensageria.falha('\n>>>> Arquivo inválido. Necessário que seja um csv')
      return

    dest = 'log_eventos.csv'
    !rm log_eventos.csv
    os.rename(src, dest)

    df = pd.read_csv(dest, sep=';', encoding='utf-8')
    self.__parse_df(df)

  def has_event_log(self):
    return not self.__event_log is None

  def has_social_log(self):
    return not self.__social_log is None

  def has_process_tree(self):
    return self.has_event_log() and not self.__process_tree is None

  def has_social_tree(self):
    return self.has_social_log() and not self.__social_tree is None

  def has_bpmn_model(self):
    return self.has_event_log() and not self.__bpmn_model is None

  def has_dfg(self):
    return self.has_event_log() and not self.__dfg is None and not self.__start_activities is None and not self.__end_activities is None

  def has_dfg_social(self):
    return self.has_social_log() and not self.__dfg_social is None and not self.__start_resources is None and not self.__end_resources is None

  def get_event_log(self):
    if not self.has_event_log():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__event_log

  def get_social_log(self):
    if not self.has_social_log():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__social_log

  def get_process_tree(self):
    if not self.has_process_tree():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__process_tree

  def get_social_tree(self):
    if not self.has_social_tree():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__social_tree

  def get_bpmn_model(self):
    if not self.has_bpmn_model():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__bpmn_model

  def get_dfg(self):
    if not self.has_dfg():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__dfg

  def get_dfg_social(self):
    if not self.has_dfg_social():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__dfg_social

  def get_start_activities(self):
    if not self.has_dfg():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__start_activities

  def get_end_activities(self):
    if not self.has_dfg():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__end_activities

  def get_start_resources(self):
    if not self.has_dfg_social():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__start_resources

  def get_end_resources(self):
    if not self.has_dfg_social():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    return self.__end_resources

  def inspect(self):
    if not self.has_event_log():
      mensageria.falha('\n>>>> NÃO HÁ DADOS CARREGADOS')
      return
    print()
    print('Número de eventos: {}\nNúmero de processos: {}'.format(self.__num_events, self.__num_cases))
    print()
    display(HTML(self.__event_log.head().to_html()))

  def inspect_start_end_activities(self):
    print()
    print("Atividades iniciais: {}".format(self.get_start_activities()))
    print()
    print("Atividades finais: {}".format(self.get_end_activities()))

  def inspect_start_end_resources(self):
    print()
    print("Recursos iniciais: {}".format(self.get_start_resources()))
    print()
    print("Recursos finais: {}".format(self.get_end_resources()))

  def discover_bpmn_model(self):
    bpmn_model = self.get_bpmn_model()
    pm4py.view_bpmn(bpmn_model)

  def discover_process_map(self):
    pm4py.view_dfg(self.get_dfg(), self.get_start_activities(), self.get_end_activities())

  def discover_social_map(self):
    pm4py.view_dfg(self.get_dfg_social(), self.get_start_resources(), self.get_end_resources())

print()
mensageria.sucesso('\n>>>> Ferramentas carregadas com sucesso!')

In [ ]:
#@title 2º PASSO) **CARREGAR DADOS** { vertical-output: true }

##
## Instanciação
##
pm_tools = PMTools.instance()
pm_tools.load()
pm_tools.inspect()
pm_tools.inspect_start_end_activities()
pm_tools.inspect_start_end_resources()


In [ ]:
#@title 3º PASSO) **MOSTRAR MAPA DE PROCESSO**

pm_tools.discover_process_map()

In [ ]:
#@title 4º PASSO) **MOSTRAR MAPA SOCIAL**

pm_tools.discover_social_map()

# **Avançado**

In [ ]:
#@title **MOSTRAR BPMN**

pm_tools.discover_bpmn_model()

# **MiniConf**

In [ ]:
!pip install pm4py

import pandas as pd
import numpy as np
import graphviz
import pm4py

from IPython.display import display, HTML, SVG

In [ ]:
def to_datetime(date_string):
  if not isinstance(date_string, str):
    return date_string
  if not date_string:
    return date_string
  date, time = date_string.split(' ')
  date = date.split('/')[::-1]
  date = '-'.join(date)
  return f'{date} {time}'

df_tarefas = pd.read_csv("Log_Eventos_tarefas.csv", sep=';', encoding='utf-8')
df_tarefas['timestamp'] = df_tarefas['timestamp'].apply(to_datetime)
df_tarefas['out'] = df_tarefas['out'].apply(to_datetime)
log_tarefas = pm4py.format_dataframe(df_tarefas, case_id='case_id', activity_key='activity', timestamp_key='timestamp')
sn_tarefas = pm4py.format_dataframe(df_tarefas, case_id='case_id', activity_key='resource', timestamp_key='timestamp')


print('*** Analisando o log de tarefas...')
print()
print(f'Número de processos: {len(df_tarefas.case_id.unique())}')
print(f'Número de tarefas: {len(df_tarefas)}')
display(HTML(log_tarefas.head().to_html()))

df_movimentos = pd.read_csv("Log_Eventos_movimentos.csv", sep=';', encoding='utf-8')
df_movimentos['in'] = df_movimentos['timestamp'].apply(to_datetime)
log_movimentos = pm4py.format_dataframe(df_movimentos, case_id='case_id', activity_key='activity', timestamp_key='timestamp')

print()
print('*** Analisando o log de movimentos...')
print()
print(f'Número de processos: {len(df_movimentos.case_id.unique())}')
print(f'Número de movimentos: {len(df_movimentos)}')
display(HTML(log_movimentos.head().to_html()))


In [ ]:
print('Mapa das tarefas...')
dfg_tarefas, start_tarefas, end_tarefas = pm4py.discover_dfg(log_tarefas)
pm4py.view_dfg(dfg_tarefas, start_tarefas, end_tarefas)

In [ ]:
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.statistics.traces.generic.log import case_statistics
from pm4py.statistics.traces.generic.log import case_arrival
from pm4py.statistics.sojourn_time.log import get as soj_time_get
from pm4py.statistics.concurrent_activities.log import get as conc_act_get
import json 
from functools import reduce

def seconds_to_days(seconds):
  return seconds / (60 * 60 * 24)

print('Estatísticas tarefas...')

all_case_durations = case_statistics.get_all_case_durations(log_converter.apply(log_tarefas, variant=log_converter.Variants.TO_EVENT_LOG), parameters={case_statistics.Parameters.TIMESTAMP_KEY: 'time:timestamp'})
all_case_durations = [ seconds_to_days(x) for x in all_case_durations ]

median_case_duration = case_statistics.get_median_caseduration(log_converter.apply(log_tarefas, variant=log_converter.Variants.TO_EVENT_LOG), parameters={case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"})
median_case_duration = seconds_to_days(median_case_duration)

average_case_duration = sum(all_case_durations) / len(all_case_durations)

soj_time = soj_time_get.apply(log_tarefas, parameters={soj_time_get.Parameters.TIMESTAMP_KEY: "time:timestamp", soj_time_get.Parameters.START_TIMESTAMP_KEY: "start_timestamp"})

conc_act = conc_act_get.apply(log_tarefas, parameters={conc_act_get.Parameters.TIMESTAMP_KEY: "time:timestamp", conc_act_get.Parameters.START_TIMESTAMP_KEY: "start_timestamp"})

print(f'Duração total: {all_case_durations}')
print(f'Duração média: {average_case_duration}')
print(f'Duração mediana: {median_case_duration}')
print('Tempo de permanência:')
print(json.dumps(soj_time, indent=2))
print('Atividades concorrentes:')
for key, value in conc_act:
  print(key, '->', value)



In [ ]:
print('Gráficos')
print()
print('Events distribution')
pm4py.view_events_distribution_graph(log_tarefas, distr_type="days_week", format="png")

In [ ]:
print('Rede social tarefas...')
dfg_sn_tarefas, start_sn_tarefas, end_sn_tarefas = pm4py.discover_dfg(sn_tarefas)
pm4py.view_dfg(dfg_sn_tarefas, start_sn_tarefas, end_sn_tarefas)

In [ ]:
print('BPMN "real" das tarefas...')
arvore_tarefas = pm4py.discover_process_tree_inductive(log_tarefas)
bpmn_model = pm4py.convert_to_bpmn(arvore_tarefas)
pm4py.view_bpmn(bpmn_model)


In [ ]:
print('Exportando para o Bizagi...')
pm4py.write_bpmn(bpmn_model, 'tarefas.bpmn', enable_layout=True)


In [ ]:
print('Mapa das movimentos...')
dfg_movimentos, start_movimentos, end_movimentos = pm4py.discover_dfg(log_movimentos)
pm4py.view_dfg(dfg_movimentos, start_movimentos, end_movimentos)